In [ ]:
!pip install -q imageio
!pip install -q opencv-python
!pip install git+https://github.com/tensorflow/docs

  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-rqgr69pz
  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/docs /tmp/pip-req-build-rqgr69pz
  Resolved https://github.com/tensorflow/docs to commit 6680535155460f7eb0d2d615b9749a0cf721d4ec
  Preparing metadata (setup.py) ... done
  Created wheel for tensorflow-docs: filename=tensorflow_docs-2024.5.3.31743-py3-none-any.whl size=182531 sha256=5c313451eb5b703e0db595c4c8896eaee4461edb42985e51dfd8c005d913b16c
  Stored in directory: /tmp/pip-ephem-wheel-cache-wdkhlqqw/wheels/86/0f/1e/3b62293c8ffd0fd5a49508e6871cdb7554abe9c62afd35ec53
Successfully built tensorflow-docs


In [ ]:
from absl import logging

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed

# Some modules to help with reading the UCF101 dtatset
import random
import re
import os
import tempfile
import ssl
import cv2
import numpy as np

# Some modules to display an animation using imageio
import imageio
from IPython import display

from urllib import request # requires python3

In [ ]:
logging.set_verbosity(logging.ERROR)

In [ ]:
# fetch videos from ucf101:

UCF_ROOT = "https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101"
_VIDEO_LIST = None
_CACHE_DIR = tempfile.mkdtemp()
unverified_context = ssl._create_unverified_context()

def list_ucf_videos():
  global _VIDEO_LIST
  if not _VIDEO_LIST:
    index = request.urlopen(UCF_ROOT, context=unverified_context).read().decode('utf-8')
    videos = re.findall(("v_[\w_]+\.avi"), index)
    _VIDEO_LIST = sorted(set(videos))
    return list(_VIDEO_LIST)

def fetch_ucf_video(video):
  cache_path = os.path.join(_CACHE_DIR, video)
  if not os.path.exists(cache_path):
    url_path = request.urljoin(UCF_ROOT, video)
    print("Fetching &s => %s"  % (url_path, cache_path))
    data = request.urlopen(url_path, contex=unverified_context).read()
    open(cache_path, 'wb').write(data)
  return cache_path

def crop_center_square(frame):
  x,y = frame.shape[0:2]
  min_dim = min(x,y)
  start_x =  (x//2) - (min_dim//2)
  start_y = (y//2) - (min_dim//2)
  return frame[start_x:start_x+min_dim,start_y:start_y+min_dim]

def load_video(path, max_frames=0, resize=(224,224)):
  cap = cv2.VideoCapture(path)
  frames = []
  try:
    while True:
      ret, frame = cap.read()
      if not ret:
        break
      frame = crop_center_square(frame)
      frame = cv2.resize(frame, resize)
      frame = frame[:,:,[2,1,0]]
      frames.append(frame)
      if len(frames) == max_frames:
        break
  finally:
    cap.release() # if do not release capture it will keep running in background and using memory
    return np.array(frames)/255.0

def to_gif(images):
  converted_images = np.clip(images*255,0,255).astype(np.uint8)
  imageio.mimsave('./animation.gif', converted_images, fps=25)
  return embed.embed_file('./animation.gif')

In [ ]:
!pip lis | grep url

mdurl                            0.1.2
urllib3                          2.0.7


In [ ]:
#https://github.com/google-deepmind/kinetics-i3d #kinetics github repo

In [ ]:
KINETICS_URL = "https://raw.githubusercontent.com/deepmind/kinetics-i3d/master/data/label_map.txt" # kinetics 400 label list
with request.urlopen(KINETICS_URL) as obj:
  labels = [line.decode("utf-8").strip() for line in obj.readlines()]
print("Found in total %d labels." % len(labels))

Found in total 400 labels.


In [ ]:
ucf_videos = list_ucf_videos()
ucf_videos

['v_ApplyEyeMakeup_g01_c01.avi',
 'v_ApplyEyeMakeup_g01_c02.avi',
 'v_ApplyEyeMakeup_g01_c03.avi',
 'v_ApplyEyeMakeup_g01_c04.avi',
 'v_ApplyEyeMakeup_g01_c05.avi',
 'v_ApplyEyeMakeup_g01_c06.avi',
 'v_ApplyEyeMakeup_g02_c01.avi',
 'v_ApplyEyeMakeup_g02_c02.avi',
 'v_ApplyEyeMakeup_g02_c03.avi',
 'v_ApplyEyeMakeup_g02_c04.avi',
 'v_ApplyEyeMakeup_g03_c01.avi',
 'v_ApplyEyeMakeup_g03_c02.avi',
 'v_ApplyEyeMakeup_g03_c03.avi',
 'v_ApplyEyeMakeup_g03_c04.avi',
 'v_ApplyEyeMakeup_g03_c05.avi',
 'v_ApplyEyeMakeup_g03_c06.avi',
 'v_ApplyEyeMakeup_g04_c01.avi',
 'v_ApplyEyeMakeup_g04_c02.avi',
 'v_ApplyEyeMakeup_g04_c03.avi',
 'v_ApplyEyeMakeup_g04_c04.avi',
 'v_ApplyEyeMakeup_g04_c05.avi',
 'v_ApplyEyeMakeup_g04_c06.avi',
 'v_ApplyEyeMakeup_g04_c07.avi',
 'v_ApplyEyeMakeup_g05_c01.avi',
 'v_ApplyEyeMakeup_g05_c02.avi',
 'v_ApplyEyeMakeup_g05_c03.avi',
 'v_ApplyEyeMakeup_g05_c04.avi',
 'v_ApplyEyeMakeup_g05_c05.avi',
 'v_ApplyEyeMakeup_g05_c06.avi',
 'v_ApplyEyeMakeup_g05_c07.avi',
 'v_ApplyE

In [ ]:
categories = {}
for video in ucf_videos:
  category = video[2:-12]
  if category not in categories:
    categories[category] = []
  categories[category].append(video)

print('found %d of videos in  %d overall categories' % (len(ucf_videos), len(categories)))

print("\n")
head1 = "CATEGORY"
head2 = "No. of videos"
head3 = "Details"
print(" ", head1, " \t  ", head2, " \t\t\t ", head3)
for category, sequences in categories.items():
  summary = ", ".join(sequences[:2])
  print("%-20s    %4d           %s, ..." % (category, len(sequences), summary))


found 13320 of videos in  101 overall categories


  CATEGORY  	   No. of videos  			  Details
ApplyEyeMakeup           145           v_ApplyEyeMakeup_g01_c01.avi, v_ApplyEyeMakeup_g01_c02.avi, ...
ApplyLipstick            114           v_ApplyLipstick_g01_c01.avi, v_ApplyLipstick_g01_c02.avi, ...
Archery                  145           v_Archery_g01_c01.avi, v_Archery_g01_c02.avi, ...
BabyCrawling             132           v_BabyCrawling_g01_c01.avi, v_BabyCrawling_g01_c02.avi, ...
BalanceBeam              108           v_BalanceBeam_g01_c01.avi, v_BalanceBeam_g01_c02.avi, ...
BandMarching             155           v_BandMarching_g01_c01.avi, v_BandMarching_g01_c02.avi, ...
BaseballPitch            150           v_BaseballPitch_g01_c01.avi, v_BaseballPitch_g01_c02.avi, ...
BasketballDunk           131           v_BasketballDunk_g01_c01.avi, v_BasketballDunk_g01_c02.avi, ...
Basketball               134           v_Basketball_g01_c01.avi, v_Basketball_g01_c02.avi, ...
BenchPress        

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/CV\ Course/DATA/YOLO\ OBJECT\ TRACKER/water.mp4 ./

In [ ]:
video_path = fetch_ucf_video('/content/water.mp4')
samplevid = load_video(video_path)
samplevid.shape

(475, 224, 224, 3)

In [ ]:
gif = to_gif(samplevid)

In [ ]:
gif

In [ ]:
i3d = hub.load("https://tfhub.dev/deepmind/i3d-kinetics-400/1").signatures['default']


In [ ]:
model_input = tf.constant(samplevid, dtype=tf.float32)[tf.newaxis,...]
logits = i3d(model_input)['default'][0]
logits

<tf.Tensor: shape=(400,), dtype=float32, numpy=
array([ 8.14140439e-01, -3.58421594e-01,  7.25006998e-01,  1.22594675e-02,
        3.07582736e+00, -1.98967963e-01, -3.40122104e-01, -9.27410901e-01,
       -5.61288214e+00, -1.90946937e+00, -1.01560962e+00,  3.99688005e-01,
        8.50488842e-01,  1.27405345e+00,  9.48939800e-01, -1.84399545e+00,
        4.01160687e-01, -1.32419750e-01,  6.80495143e-01, -5.33276796e+00,
       -1.02763367e+00, -1.92921293e+00, -2.22866392e+00,  7.43966579e-01,
       -1.28057301e+00,  8.21925163e-01,  1.06245852e+00,  2.81561756e+00,
       -4.37162876e+00, -5.49971640e-01,  2.07695842e-01, -1.64304364e+00,
        1.43236399e+00,  9.48918760e-01,  7.41295099e-01, -4.26083326e+00,
        1.98440826e+00, -8.91332805e-01, -4.85196495e+00, -1.45954835e+00,
       -1.55376220e+00,  8.74080539e-01,  5.88643074e+00, -3.17092061e+00,
        7.13363230e-01, -1.64805427e-01, -2.81117976e-01,  4.14741182e+00,
       -3.97002131e-01,  1.35336256e+00,  5.13542891

In [ ]:
logits.shape

TensorShape([400])

In [ ]:
probabilities = tf.nn.softmax(logits)
print('top 5 probabilities :')
for i in np.argsort(probabilities)[::-1][:5]:
  print(f"{labels[i]:22}: {probabilities[i] * 100:5.2f}%")

top 5 probabilities :
petting animal (not cat): 14.78%
sailing               : 13.68%
ironing               :  7.27%
yoga                  :  6.30%
clay pottery making   :  5.10%


In [ ]:
def predict(samplevid1):
  model_input = tf.constant(samplevid1, dtype=tf.float32)[tf.newaxis, ...]

  logits = i3d(model_input)['default'][0]
  probabilities = tf.nn.softmax(logits)

  print("Top 5 actions:")
  for i in np.argsort(probabilities)[::-1][:5]:
    print(f"  {labels[i]:22}: {probabilities[i] * 100:5.2f}%")

predict(samplevid)

Top 5 actions:
  petting animal (not cat): 14.78%
  sailing               : 13.68%
  ironing               :  7.27%
  yoga                  :  6.30%
  clay pottery making   :  5.10%


In [ ]:
def predict(samplevidpath):
  video_path = fetch_ucf_video(samplevidpath)
  samplevid1 = load_video(video_path)
  gif = to_gif(samplevid1)
  model_input = tf.constant(samplevid1, dtype=tf.float32)[tf.newaxis, ...]

  logits = i3d(model_input)['default'][0]
  probabilities = tf.nn.softmax(logits)

  print("Top 5 actions:")
  for i in np.argsort(probabilities)[::-1][:5]:
    print(f"  {labels[i]:22}: {probabilities[i] * 100:5.2f}%")

predict('/content/drive/MyDrive/CV Course/DATA/YOLO OBJECT TRACKER/Playing Chess.mp4')

Top 5 actions:
  playing chess         : 99.94%
  skateboarding         :  0.02%
  tasting beer          :  0.01%
  hurdling              :  0.01%
  drinking shots        :  0.00%
